In [5]:
import gmsh
import numpy as np

In [6]:
E=30e6 #psi
nu=0.3 
t=1 #in
tension=1000 #psi

In [7]:
gmsh.initialize()
gmsh.model.add('ej2guia3')
lc =0.5 #factor de mallado. Cuanto más chico este número, más fino es el mallado
L = 10
#definición de los puntos en gmsh y que tan fino será el mallado cerca de ellos. lc=3 es un mallado grueso. Un mallado fino se obtiene con, por ejemplo, lc=0.5
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc)
p2 = gmsh.model.geo.addPoint(L, 0, 0, lc) 
p3 = gmsh.model.geo.addPoint(L, L/2, 0, lc) 
p4 = gmsh.model.geo.addPoint(0, L/2, 0, lc) 
p5 = gmsh.model.geo.addPoint(0, 1, 0, lc/20)  #para refinar el mallado cerca del agujero, se puede dividir el factor lc por 20 (por ejemplo), para los puntos p5 y p6
p6 = gmsh.model.geo.addPoint(1, 0, 0, lc/20)  #tambien se puede refinar el mallado cerca de cualquier otro punto de interes
#definición de las lineas
l1 = gmsh.model.geo.addLine(p6, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
#definición del arco
l5 = gmsh.model.geo.addCircleArc(p5,p1,p6)
#generación de loop
C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4, l5])
#generación de superficie
S1 = gmsh.model.geo.addPlaneSurface([C1])

gmsh.model.geo.synchronize()
#gmsh.fltk.run()

gmsh.model.mesh.generate(2)

gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)
gmsh.write('ej2guia3.msh')

#gmsh.fltk.run()

In [8]:
#Condiciones de contorno, empotramiento ficticio por simetrías
EmpotradoX = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1,EmpotradoX,'EmpotradoX')
EmpotradoY = gmsh.model.addPhysicalGroup(1,[l1])
gmsh.model.setPhysicalName(1,EmpotradoY,'EmpotradoY')

Traccionado = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado')
Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')
#Nodo fuera del mallado. Se encuentra en el centro del agujero pasante. Tiene que estar empotrado para que la matriz no sea singular
NodoFicticio = gmsh.model.addPhysicalGroup(0, [p1])
gmsh.model.setPhysicalName(0, NodoFicticio, 'Ficticio')

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

In [9]:
NodeInfo = gmsh.model.mesh.get_nodes()

NumeroNodos = NodeInfo[0].shape[0]

MN = NodeInfo[1].reshape(NumeroNodos , 3) #MN= Matriz de Nodos

ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)

MC = ELEMENTS.reshape([ETAGS.shape[0],3]) #MC=Matriz de conectividad

NodoFicticio = gmsh.model.mesh.get_nodes_for_physical_group(0,NodoFicticio)[0].astype(int)-1
NodosEmpotradosX = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpotradoX)[0].astype(int)-1
NodosEmpotradosY = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpotradoY)[0].astype(int)-1

#gmsh.fltk.run()

In [10]:
entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)
Longitudes = np.abs( MN[Ltraccionada[:,0]-1,1] - MN[Ltraccionada[:,1]-1,1] )

In [11]:
#Se genera un vector con las fuerzas impuestas en el lado traccionado
Fuerzas = np.zeros((2*NumeroNodos,1))

for l, linea in enumerate(Ltraccionada):
    Flocal = np.array([[1],[1]])*tension*t*Longitudes[l]
    n1 = linea[0]
    n2 = linea[1]
    Fuerzas[ np.array([2*(n1-1), 2*(n2-1)], dtype=int)] += Flocal

In [12]:
#definición de las matrices r y s.
r = np.arange(2*NumeroNodos)
S = []

for i in NodosEmpotradosX:
    S.append(2*i)
for i in NodosEmpotradosY:
    S.append(2*i+1)
for i in NodoFicticio:
    S.append(2*i)
    S.append(2*i+1)

s = S
s = np.ravel(np.sort(s))
r = np.delete( r, s )


In [13]:
#calculando los desplazamientos y resultantes
D = np.dot((E/(1-nu**2)),np.array([[1,nu,0],[nu,1,0],[0,0,.5*(1-nu)]]))

A = []
beta = np.zeros((3,3))
gamma = np.zeros((3,3))
Matriz_area = np.zeros((3,3))
Matriz_area[:,0] = 1
kglobal = np.zeros((2*len(MN),2*len(MN)))
Btot = []

for i in range(len(MC)):
    kred = np.zeros((8,8))
    B = np.zeros((3,6))
    I = MC[i,:]-1
    X = MN[I,0]
    Y = MN[I,1]
    
    Matriz_area[:,1] = X
    Matriz_area[:,2] = Y
    
    A=(np.linalg.det(Matriz_area)/2) #determinante de la matriz de area
    #Armado de la matriz B
    beta[0] = Y[1] - Y[2]
    beta[1] = Y[2] - Y[0]
    beta[2] = Y[0] - Y[1]
    gamma[0] = X[2] - X[1]
    gamma[1] = X[0] - X[2]
    gamma[2] = X[1] - X[0]
    for j in range(3):
        B[0,2*j] = beta[j,0]
        B[1,2*j + 1] = gamma[j,0]
        B[2,2*j] = gamma[j,0]
        B[2,2*j + 1] = beta[j,0]
    B = B/(2*A)
    Btot.append(B)
    #calculo y ensamble de la matriz de rigidez global
    kloc = t*np.absolute(A)*np.dot(np.transpose(B),np.dot(D,B))
    A_G = np.array([int(I[0]*2),int(I[0]*2+1),int(I[1]*2),int(I[1]*2+1),int(I[2]*2),int(I[2]*2+1)]) 
    kglobal[np.ix_(A_G,A_G)] += kloc

In [14]:
#Calculo desplazamientos y resultantes
kglobal_r = kglobal[np.ix_(r,r)] #matriz de rigidez global en los nodos no empotrados
desp = (np.dot(np.linalg.inv(kglobal_r),Fuerzas[r])) 
Desplazamientos = np.zeros((2*len(MN),1))
a = 0

for i in r:
    Desplazamientos[i] += desp[a]
    a += 1
#A las resultantes les sumo las fuerzas impuestas como condicion
Resultantes = np.zeros((2*len(MN),1))    
Resultantes[s] = np.dot(kglobal[s,:],Desplazamientos)
Resultantes = Resultantes+Fuerzas
Desplazamientos
MC = MC-1

In [15]:
elementos= MC.shape[0]
nodos = MN.shape[0]
#Calculo de las tensiones
Tensiones=[]
for i in range(elementos):
    N = MC[i,:]
    IndicA = []
    Indic = []
    for j in range(3):
        IndicA.append(np.linspace(2*N[j], (2*N[j] + 1), 2))
    Indic = np.ravel(IndicA).astype(int)
    Tensiones.append(np.dot(D, np.dot(Btot[i], Desplazamientos[Indic])))
TensionesPromM = np.zeros((nodos,2))
Tensiones = np.array(Tensiones)
TensionesProm = np.zeros(nodos)
#Promediado de las tensiones
for i in range(elementos):
    N = MC[i,:]
    TensionesPromM[N[0],0] += Tensiones[i,0]
    TensionesPromM[N[1],0] += Tensiones[i,0]
    TensionesPromM[N[2],0] += Tensiones[i,0]
    #TensionesPromM[:,0] suma las tensiones de todos los elementos que comparten el mismo nodo
    TensionesPromM[N[0],1] += 1
    TensionesPromM[N[1],1] += 1
    TensionesPromM[N[2],1] += 1
    #TensionesPromM[:,1] suma cuantos elementos comparten el mismo nodo
Desplazamientos_=np.zeros((MN.shape[0],3))
Resultantes_=np.zeros((MN.shape[0],3))


for i in range(MN.shape[0]):
    if i not in NodoFicticio:  #Deja de lado los nodos que no forman parte real del mallado, sino que se ponen como condición de vínculo
        TensionesProm[i] = TensionesPromM[i,0]/TensionesPromM[i,1] #Promedia la tension por la cantidad de elementos que rodean al nodo
        #Despliego el array en 2 columnas
        Desplazamientos_[i,0]=Desplazamientos[2*i]
        Resultantes_[i,0]=Resultantes[2*i]
        Desplazamientos_[i,1]=Desplazamientos[2*i+1]
        Resultantes_[i,1]=Resultantes[2*i+1]
        

In [16]:
desps = gmsh.view.add("Desplazamientos [in]")
Desps = gmsh.view.addModelData(desps, 0, 'ej2guia3', 'NodeData', NodeInfo[0], Desplazamientos_, numComponents=3)
resultantes = gmsh.view.add("Resultantes [lbf]")
Resultantes = gmsh.view.addModelData(resultantes, 0, 'ej2guia3', 'NodeData', NodeInfo[0], Resultantes_, numComponents=3)
TensionesProm_ = gmsh.view.add("Tensiones Promedio [psi]")
TensionesProm = gmsh.view.addModelData(TensionesProm_, 0, 'ej2guia3', 'NodeData', NodeInfo[0], TensionesProm.reshape(-1,1), numComponents=1)
Tensiones_ = gmsh.view.add("Tensiones [psi]")
Tensiones = gmsh.view.addModelData(Tensiones_,0,'ej2guia3','ElementData',ETAGS,Tensiones[:,0].reshape(-1,1),numComponents=1)

In [1]:
#Representacion rápida de la posicion de los nodos
import matplotlib.pyplot as plt
from matplotlib import quiver

plt.style.use('default')
plt.rc('figure',figsize=(15,10))

plt.plot(MN[:,0],MN[:,1],'ok')


In [ ]:
gmsh.fltk.run()
gmsh.finalize()